# Deep Knowledge Tracing Plus (LPKT)

This notebook will show you how to train and use the LPKT.
First, we will show how to get the data (here we use assistment-2017 as the dataset).
Then we will show how to train a LPKT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [LPKT.py](LPKT.ipynb)


## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
import numpy as np
from load_data import DATA

def generate_q_matrix(path, n_skill, n_problem, gamma=0):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            problem2skill = eval(line)
    q_matrix = np.zeros((n_problem + 1, n_skill + 1)) + gamma
    for p in problem2skill.keys():
        q_matrix[p][problem2skill[p]] = 1
    return q_matrix

batch_size = 64
n_at = 9632
n_it = 2890
n_question = 102
n_exercise = 3162
seqlen = 500
d_k = 128
d_a = 50
d_e = 128
q_gamma = 0.3
dropout = 0.2

dat = DATA(seqlen=seqlen, separate_char=',')
train_data = dat.load_data('../../data/anonymized_full_release_competition_dataset/train.txt')
test_data = dat.load_data('../../data/anonymized_full_release_competition_dataset/test.txt')
q_matrix = generate_q_matrix(
    '../../data/anonymized_full_release_competition_dataset/problem2skill',
    n_question, n_exercise,
    q_gamma
)

## Training and Persistence

In [2]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [3]:
from EduKTM import LPKT

lpkt = LPKT(n_at, n_it, n_exercise, n_question, d_a, d_e, d_k, q_matrix, batch_size, dropout)
lpkt.train(train_data, test_data, epoch=2)
lpkt.save("lpkt.params")

Testing:   0%|          | 0/13 [00:00<?, ?it/s]

[Epoch 0] LogisticLoss: 0.747071


Training:   0%|          | 0/31 [00:00<?, ?it/s]

[Epoch 0] auc: 0.647351, accuracy: 0.624059


Testing:   0%|          | 0/13 [00:00<?, ?it/s]

[Epoch 1] LogisticLoss: 0.628299


Testing: 100%|██████████| 13/13 [01:20<00:00,  6.20s/it]
INFO:root:save parameters to lpkt.params


[Epoch 1] auc: 0.663860, accuracy: 0.670790


## Loading and Testing

In [4]:
lpkt.load("lpkt.params")
_, auc, accuracy = lpkt.eval(test_data)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from lpkt.params
Testing: 100%|██████████| 13/13 [01:20<00:00,  6.23s/it]

auc: 0.670790, accuracy: 0.663860
